# Python Średniozaawansowany

#### Instrukcja instalacji VS Code <a class="anchor" id="install"></a>

 - Zainstaluj narzędzia [Python](https://www.python.org/downloads/).

 - Pobierz [VS Code](https://code.visualstudio.com/).

 - Zainstaluj rozszerzenie [Python](https://marketplace.visualstudio.com/items?itemName=ms-python.python).

 - Zainstaluj rozszerzenie [Jupyter](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter).

  - Zainstaluj rozszerzenie [Git Extension Pack](https://marketplace.visualstudio.com/items?itemName=donjayamanne.git-extension-pack).

#### Potrzebne importy

 - [matplotlib](https://matplotlib.org/) - wymagana instalacja
    * [pyplot](https://matplotlib.org/stable/tutorials/introductory/pyplot.html)
 - [numpy](https://numpy.org/) - wymagana instalacja
 - os
 - [inspect](https://docs.python.org/3/library/inspect.html) - biblioteka zawierająca [funkcjonalność przydatną przy analizowaniu struktur klas](https://docs.python.org/3/library/inspect.html#classes-and-functions).
 - [abc](https://docs.python.org/3/library/abc.html) - biblioteka do abstrakcji klasowej


In [1]:
from abc import ABC, abstractmethod
import traceback
import os
from inspect import getmro

In [2]:
from matplotlib import pyplot as plt
import numpy as np

## Wyjątki

### [Wyjątki](https://en.wikipedia.org/wiki/Exception_handling) są zdarzeniami, które pojawiają się podczas wykonania programu i przerywają jego normalny przebieg. Technicznie są to obiekty reprezentujące dany błąd.

In [3]:
a = 5
b = 0
print(a/b)

ZeroDivisionError: division by zero

##### Wszystkie wbudowane, nie opuszczające systemu [wyjątki](https://docs.python.org/3/library/exceptions.html) dziedziczą po klasie `Exception`. Wyjątki tworzone przez programistę również powinny dziedziczyć po tej klasie.

##### Najczęściej spotykane wyjątki to

 - `AssertionError` - niespełnienie warunku operacji [assert](https://docs.python.org/3/reference/simple_stmts.html#the-assert-statement)
 - `AttributeError` - odwołanie do nieistniejącego atrybutu
 - `IOError` - błąd operacji wejścia/wyjścia
 - `IndexError` - odwołanie do kolekcji za pomocą indeksu poza zakresem
 - `ImportError` - problem z załadowaniem modułu
 - `KeyError` - odwołanie do słownika nieistniejącym kluczem
 - `ValueError` - przekazanie do funkcji argumentu o niepoprawnej wartości
 - `ZeroDivisionError` - dzielenie przez zero

### Obsługa wyjątków

##### Wyjątki obsługujemy za pomocą słów kluczowych:
 - `try` - poprzedza obszar sprawdzany pod kątem wyrzucania wyjątków,
 - `except` - poprzedza obszar obsługi wyłapanego wyjątku,
 - `finally` - poprzedza obszar, który jest wykonywany pod koniec obsługi, niezależnie od tego, czy wyjątek został wyrzucony czy też nie
 - `raise` - wyrzuca zadany wyjątek

In [4]:
for i in [0.5, 0.3, 0]:
    result = 0
    try:
        if i == 0:
            result = 10000000
        else:
            result = 10/i
    except:
        result = 10000000
    print(i, " ", result)

0.5   20.0
0.3   33.333333333333336
0   10000000


In [5]:
def root4(x):
    return x ** 0.25

x = "Ala ma kota"

try:
    print(root4(x))
except:
    print("Hej nie umiem tego potęgować")

Hej nie umiem tego potęgować


In [6]:
for i in [0.5, 0.3, 0.001]:
    result = 0
    try:
        result = 10/(max(0.1,i) - 0.1)
    except:
        result = 10000000
        raise ValueError("Value lower than 0.1")
    finally:
        print(i, " ", result)


0.5   25.0
0.3   50.00000000000001
0.001   10000000


ValueError: Value lower than 0.1

##### Do każdego bloku `try` możemy przypisać wiele bloków `except`.

In [ ]:
l = [5, 0, 6, 2]

for i in range(5):
    try:
        a = 1/l[i]
        print(a)
        assert a >= 0.2
    except KeyError:
        print("Wrong key")
    except (ZeroDivisionError):
        print("Oopsie, whoopsie. You tried to divide by zero")
    except (IndexError, KeyError):
        print("I went too far")
    except:
        print("What did just happened here?")

##### Klauzulę `finally` zazwyczaj używamy, gdy po wykonaniu zadania musimy zwolnić pewne zasoby, niezależnie, czy procedura powiodła się czy nie. Na przykład, obsługa pliku.

In [7]:
dummy_file_name = "dummy.file"

try:
    file = open(dummy_file_name, "r")
    print(file.read())
except:
    print("File is not readable")
finally:
    file.close()
    if os.path.exists(dummy_file_name):
        os.remove(dummy_file_name)


File is not readable


NameError: name 'file' is not defined

### Manager kontekstu

##### Zamiast tego możemy użyć wyrażenia [with](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) zwanego managerem kontekstu, które obsługuje obiekty posiadające metody `__enter__` i `__exit__`

In [8]:
dummy_file_name = "dummy.file"

with open(dummy_file_name, "w") as file:
    file.write("Alamakotaasierotkamarysia")

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

with open(dummy_file_name, "r") as file:
    print(file.read())

FileNotFoundError: [Errno 2] No such file or directory: 'dummy.file'

In [9]:
dummy_file_name = "dummy.file"

class AutodestructingFile:

    def __init__(self, address) -> None:
        self.address = address
        self.writer = open(address, "w")
        self.reader = open(address, "r")

    def write(self, text):
        self.writer.write(text)

    def read(self):
        return self.reader.read()

    def close(self):
        self.reader.close()
        self.writer.close()
        if os.path.exists(self.address):
            os.remove(self.address)

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        self.close()


with AutodestructingFile(dummy_file_name) as file:
    file.write("lol")
    print(file.read())

    raise Exception("lol")
    print("message ignored")

Exception: lol

In [ ]:
from contextlib import contextmanager

dummy_file_name = "dummy.file"

class ReadWriteFile:
    def __init__(self, address) -> None:
        self.address = address
        self.writer = open(address, "w")
        self.reader = open(address, "r")

    def write(self, text):
        self.writer.write(text)

    def read(self):
        return self.reader.read()


@contextmanager
def autodestructingFile(address):
    file = ReadWriteFile(address)
    yield file
    file.reader.close()
    file.writer.close()
    if os.path.exists(address):
        os.remove(address)


with autodestructingFile(dummy_file_name) as file:
    file.write("lol")
    print(file.read())

    raise Exception("lol")
    print("message ignored")

### [Rzucanie wyjątków](https://docs.python.org/3/tutorial/errors.html#raising-exceptions)

##### W celu rzucenia wyjątku wykorzystujemy wyrażenie `raise`.

In [ ]:
raise Exception("My random exception")

### Tworzenie własnych wyjątków

In [10]:
class CustomException1(Exception):
    pass

try:
    raise CustomException1("Some custom exception")
except:
    traceback.print_exc()

class CustomException2(Exception):
    def __init__(self) -> None:
        super().__init__("My hardcoded message")

    def sing(self):
        print("In the marry month of June")

try:
    raise CustomException2()
except CustomException2 as e:
    traceback.print_exc()
    e.sing()

In the marry month of June


Traceback (most recent call last):
  File "C:\Users\micha\AppData\Local\Temp/ipykernel_16440/2871786223.py", line 5, in <module>
    raise CustomException1("Some custom exception")
CustomException1: Some custom exception
Traceback (most recent call last):
  File "C:\Users\micha\AppData\Local\Temp/ipykernel_16440/2871786223.py", line 17, in <module>
    raise CustomException2()
CustomException2: My hardcoded message


In [11]:
def fact(n):
    if n == 0:
        return 1
    else:
        return fact(n - 1) * n

In [12]:
np.array([2]) ** 31

array([-2147483648], dtype=int32)

## [Programowanie obiektowe](https://en.wikipedia.org/wiki/Object-oriented_programming)

##### Podstawowe koncepcje to

 - [dziedziczenie](https://en.wikipedia.org/wiki/Inheritance_(object-oriented_programming)),
 - [kompozycja](https://en.wikipedia.org/wiki/Object_composition),
 - [abstrakcja](https://en.wikipedia.org/wiki/Abstraction_(computer_science)),
 - [hermetyzacja](https://en.wikipedia.org/wiki/Encapsulation_(computer_programming)),
 - [polimorfizm](https://en.wikipedia.org/wiki/Polymorphism_(computer_science))

### Dziedziczenie

Jeżeli pewna klasa posiada funkcjonalność, która jest nam potrzebna, ale chcielibyśmy jeszcze dodać coś od siebie, możemy rozszerzyć klasę tworząc nową, która dziedziczy funkcjonalności klasy bazowej.

In [13]:
class Animal:
    def __init__(self, name) -> None:
        self.name = name

    def eat(self, food):
        print(self.name, ":Nom, nom, nom. Tasty", food)

class Bird(Animal):
    def fly(self):
        print(self.name, ": I can fly")

animal = Animal("Creature")
animal.eat("other animal")

bird = Bird("Tweety")
bird.eat("grain")
bird.fly()

def feed(animal: Bird, food: str) -> None:
    animal.eat(food)
    animal.fly()

feed(bird, "grain")
bird.__str__()

Creature :Nom, nom, nom. Tasty other animal
Tweety :Nom, nom, nom. Tasty grain
Tweety : I can fly
Tweety :Nom, nom, nom. Tasty grain
Tweety : I can fly


'<__main__.Bird object at 0x000001931EF26AF0>'

Możemy sprawdzić pochodzenie naszej klasy za pomocą funkcji `getmro` z pakietu `inspect`.

In [14]:
print(getmro(type(bird)))
print(getmro(type(getmro)))
print(getmro(AttributeError))
print(getmro(type(5.0)))

(<class '__main__.Bird'>, <class '__main__.Animal'>, <class 'object'>)
(<class 'function'>, <class 'object'>)
(<class 'AttributeError'>, <class 'Exception'>, <class 'BaseException'>, <class 'object'>)
(<class 'float'>, <class 'object'>)


Możemy też dziedziczyć po wielu klasach przejmując atrybuty każdej z nich.

In [15]:
class Animal:
    def __init__(self, name) -> None:
        self.name = name
        print("I am an animal")

    def eat(self, food):
        print(self.name, ": Nom, nom, nom. Tasty", food)

class Oviparous(Animal):
    def __init__(self, name) -> None:
        super().__init__(name)
        print("I am an oviparous")

    def bearEgg(self):
        print(self.name, ": plop")

    def breath(self):
        print(self.name, ": wheez")

class Mammal(Animal):
    def __init__(self, name) -> None:
        super().__init__(name)
        print("I am a mammal")

    def giveMilk(self):
        print(self.name, ": blurp")

    def breath(self):
        print(self.name, ": phew")

class Platypus(Oviparous, Mammal):
    def __init__(self, name) -> None:
        super().__init__(name)
        print("I am a platypus")

    def sting(self):
        print(self.name, ": sting")

perry = Platypus("Perry")

perry.bearEgg()
perry.giveMilk()
perry.eat("algae")
perry.breath()

print(getmro(Platypus))

I am an animal
I am a mammal
I am an oviparous
I am a platypus
Perry : plop
Perry : blurp
Perry : Nom, nom, nom. Tasty algae
Perry : wheez
(<class '__main__.Platypus'>, <class '__main__.Oviparous'>, <class '__main__.Mammal'>, <class '__main__.Animal'>, <class 'object'>)


In [16]:
class Base1:
    def __init__(self) -> None:
        self.id = 1

    def printId(self):
        print(self.id)

class Base2:
    def __init__(self) -> None:
        self.name = "2"

    def printName(self):
        print("It is me, number", self.name)

class Child(Base1, Base2):
    def __init__(self) -> None:
        super().__init__()
        self.id = 2
        self.name = "1"

c = Child()
c.printId()
c.printName()

print(getmro(Child))

2
It is me, number 1
(<class '__main__.Child'>, <class '__main__.Base1'>, <class '__main__.Base2'>, <class 'object'>)


Zauważmy, że powyższa klasa rozwiązuje dziedziczenie wpierw rozpatrując klasę `Base1`. Oznacza to, że jeśli klasy `Base1` i `Base2` będą miały takie samo pole, `Child` przejmie tylko pole od `Base1`.

In [17]:
class Base1:
    def __init__(self) -> None:
        self.id = 1

    def printId(self):
        print(self.id)

class Base2:
    def __init__(self) -> None:
        self.id = 2

    def printId(self):
        print("It is me, number", self.id)

class Child1(Base1, Base2):
    def __init__(self) -> None:
        super().__init__()
        self.id = 2

class Child2(Base2, Base1):
    def __init__(self) -> None:
        super().__init__()
        self.id = 1

c1 = Child1()
c2 = Child2()

c1.printId()
c2.printId()

print(getmro(Child1))
print(getmro(Child2))

2
It is me, number 1
(<class '__main__.Child1'>, <class '__main__.Base1'>, <class '__main__.Base2'>, <class 'object'>)
(<class '__main__.Child2'>, <class '__main__.Base2'>, <class '__main__.Base1'>, <class 'object'>)


### Kompozycja

Zamiast dziedziczyć wszystkie atrybuty możemy wkomponować jedną klasę w drugą tworząc obiekt tej pierwszej jako pole drugiej. Następnie pole to możemy wykorzystać do implementacja własnych metod.

### Abstrakcja

Czasami chcemy stworzyć wzór klasy bez implementowania poszczególnych jej elementów. Przkładowo, chcę stworzyć klasę `Collection`, która pozwoli użytkownikowi na dostęp do elementów, ale pozwoli również zaimplementować różne kontenery przechowujące elementy.

Nie jest to podstawowy koncept w Pythonie ze względu na brak [silnej typizacji](https://en.wikipedia.org/wiki/Strong_and_weak_typing), stąd musimy wykorzystać do tego pakiet `abc`.

In [18]:
class Collection(ABC):
    @abstractmethod
    def __init__(self, elements) -> None:
        pass

    @abstractmethod
    def getNext(self):
        pass

class SingleElement(Collection):
    def __init__(self, elements) -> None:
        self.element = elements

    def getNext(self):
        return self.element

class Array(Collection):
    def __init__(self, elements) -> None:
        self.elements = elements
        self.i = 0

    def getNext(self):
        element = self.elements[self.i]
        self.i %= len(self.elements)
        return element

Niektóre pola nie powinny być dostępne z zewnątrz. Jeżeli nasza klasa wykonuje jakąś operację na własnych parametrach możemy chcieć chronić te parametry przed zmianą z zewnątrz lub przez klasy dziedziczące.

Pola [chronione](https://en.wikipedia.org/wiki/Access_modifiers) nie są dostępne użytkownikom z zewnątrz, aczkolwiek są dostępne dla klas dziedziczących. Pola takie zapisujemy z pojedyńczym podkreślnikiem (`_`) przed nazwą.

## Dekoratory

## Wyrażenia lambda

## Generatory i iteratory

## Wyrażenia regularne

## Wielowątkowość

## Profilowanie kodu